## Class Scheduling Problem


####  Step 0: The model

Decision Variables:

$$\begin{align*}
&I \in [{1,2,3,4,5,6,7,8,9,10}]\\
&K \in [{1,2,3,4,5,6,7}]\\
&M \in [{1,2,3,4}]\\
\end{align*}$$

$X_{ki} = \begin{cases}
1 & \mbox{if class $k \in K$ is in room $i \in I$,} \\
0 & \mbox{otherwise.}
\end{cases}$

$Z_{kij} = \begin{cases}
1 & \mbox{if class $k \in K$ is moving from room $i \in I$ to room $j \in $I ,} \\
0 & \mbox{otherwise.}
\end{cases}$

$$\begin{array}{rll}
\text{min} & \displaystyle 3Y_1^+ + 2Y_2^+ + 2Y_4^+\\
\text{s.t.}\\
% Goal Optimization Goals
& \text{(1) }\displaystyle \sum_{i=1}^I X_{ki}R_{i} +\sum_{i=1}^I\sum_{j=1}^I Z_{kij}TE_{kij} - (Y_{1}^+ - Y_{1}^-) = 1100 \quad \forall k\in K\\
& \text{(2) }\displaystyle \sum_{i=1}^I X_{ki}R_{i} - (Y_{2}^+ - Y_{2}^-) = 19 \quad \forall k\in K\\
& \text{(3) }\displaystyle \sum_{i=1}^I\sum_{j=1}^I X_{ki}Z_{kij}T_{kij} - (Y_{3}^+ - Y_{3}^-) = 55 \quad \forall k\in K\\
& \text{(4) }\displaystyle \sum_{i=1}^I\sum_{j=1}^I X_{ki}Z_{kij}E_{kij} - (Y_{4}^+ - Y_{4}^-) = 19 \quad \forall k\in K\\
% Constraints
& \text{(5) }\displaystyle \sum_{i=1}^I\sum_{j=1}^I Z_{kij} \geq 5 \quad \forall k\in K\\
& \text{(6) }\displaystyle \sum_{i=9}^I X_{ki} \leq 2 \quad \forall k\in K\\
& \text{(7) }\displaystyle \sum_{i=1}^I X_{ki} = 1 \quad \forall k\in K\\
& \text{(8) }\displaystyle X_{k8} = 1 \quad \forall k\in K\\
\end{array}$$

$$\begin{align*}
&T_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&E_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I,\text{and } i \neq j \\
&TE_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&R_{i} \geq 0 \text{ and integer where } k \in K, i \in I \\
&X_{ki} \in [{0,1}] \text{ where } k \in K, i \in I \\
&Z_{kij} \in [{0,1}] \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&Y_{m}^+,Y_{m}^- \geq 0 \text{ where } m \in M, \text{and } Y_{m}^+ \text{ is a surplus variable and } Y_{m}^- \text{is a slack variable }\\
\end{align*}$$

#### Step 1: Import gurobipy module

In [7]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

#### Step 1.1 Reading from a file

In [8]:
MainFile = pd.read_json("Testing.txt",orient='columns')
MainFile

dataframe = pd.DataFrame(MainFile)
print(dataframe)
 
# print(dataframe['Average Transition Time Matrix']) 
# print(dataframe['Transition Risk Exposure Score']) 
# print(dataframe['Room Risk Exposure Score']) 

# dataframe['Average Transition Time Matrix'][0][0] # grabbing values from matrix ['Average Transition Time Matrix'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Transition Risk Exposure Score'][0][0] # grabbing values from matrix ['Transition Risk Exposure Score'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Room Risk Exposure Score']['Score'][0] # grabbing values from matrix ['Room Risk Exposure Score']['Score'][i] where i={0-9}


           Average Transition Time Matrix  Transition Risk Exposure Score  \
1      [0, 5, 6, 7, 8, 9, 10, 11, 12, 13]  [0, 4, 4, 4, 3, 3, 3, 2, 2, 3]   
2       [5, 0, 5, 6, 7, 8, 9, 10, 11, 12]  [4, 0, 4, 4, 3, 3, 3, 2, 2, 3]   
3        [6, 5, 0, 5, 6, 7, 8, 9, 10, 11]  [4, 4, 0, 4, 3, 3, 3, 2, 2, 3]   
4         [7, 6, 5, 0, 5, 6, 7, 8, 9, 10]  [4, 4, 4, 0, 3, 3, 3, 2, 2, 3]   
5          [8, 7, 6, 5, 0, 5, 6, 7, 8, 9]  [3, 3, 3, 3, 0, 4, 4, 2, 2, 1]   
6          [9, 8, 7, 6, 5, 0, 5, 6, 7, 8]  [3, 3, 3, 3, 4, 0, 4, 2, 2, 1]   
7         [10, 9, 8, 7, 6, 5, 0, 5, 6, 7]  [3, 3, 3, 3, 4, 4, 0, 2, 2, 1]   
8        [11, 10, 9, 8, 7, 6, 5, 0, 5, 6]  [2, 2, 2, 2, 2, 2, 2, 0, 2, 2]   
9       [12, 11, 10, 9, 8, 7, 6, 5, 0, 5]  [2, 2, 2, 2, 2, 2, 2, 2, 0, 2]   
10     [13, 12, 11, 10, 9, 8, 7, 6, 5, 0]  [3, 3, 3, 1, 1, 1, 1, 2, 2, 0]   
Score                                 NaN                             NaN   

             Room Risk Exposure Score  
1                                 N

#### Step 2: Define the model

In [9]:
m = gp.Model()

#### Step 3: Define your sets

In [10]:
# number of rooms in the system
I = 10
# number of classes in the system
K = 7
# number of goals we have for the system
M = 4

#### Step 4: Define the parameters

In [11]:
# dataframe['Average Transition Time Matrix'][0][0] # grabbing values from matrix ['Average Transition Time Matrix'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Transition Risk Exposure Score'][0][0] # grabbing values from matrix ['Transition Risk Exposure Score'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Room Risk Exposure Score']['Score'][0] # grabbing values from matrix ['Room Risk Exposure Score']['Score'][i] where i={0-9}

mPlus = [3,2,2,2]
mMinus = [0,0,0,0]

#### Step 5: Define the decision variables

In [12]:
x = {}
for k in range(K):
    for i in range(I):
        x[k,i] = m.addVar(vtype=GRB.BINARY, name="x_"+str(k)+str(i))
z = {}
for i in range(I):
    for j in range(I):
        z[k,i,j] = m.addVar(vtype=GRB.BINARY, name="z_"+str(k)+str(i)+str(j))
    
yPlus = {}
yMinus = {}
for i in range(M):
    yPlus[i] = m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, obj=mPlus[i] , name="y+_"+str(i))
    yMinus[i]= m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, obj=mMinus[i] , name="y-_"+str(i))


#### Step 6: Set the objective function


In [13]:
m.modelSense = GRB.MINIMIZE

#### Step 6: Add the constraints

In [24]:
# Constraint (1)
m.addConstrs(((sum(x[k,i]*dataframe['Room Risk Exposure Score']['Score'][i]) for i in range(I)) + \
              (sum(sum(z[k,i,j]*dataframe['Average Transition Time Matrix'][i][j]*dataframe['Transition Risk Exposure Score'][i][j]) for j in range(I)) for i in range(I)) + \
              (yPlus[0]-yMinus[0]) == 1100) for k in range(K))

# Constraint (2)
m.addConstrs(((sum(x[k,i]*dataframe['Room Risk Exposure Score']['Score'][i])) for i in range(I) + \
              (yPlus[1]-yMinus[1]) == 19) for k in range(K))

# Constraint (3)
m.addConstrs(((sum(sum(x[k,i]*z[k,i,j]*dataframe['Average Transition Time Matrix'][i][j]) for j in range(I)) for i in range(I)) + \
              (yPlus[2]-yMinus[2]) == 55) for k in range(K)) 

# Constraint (4)
m.addConstrs(((sum(sum(x[k,i]*z[k,i,j]*dataframe['Transition Risk Exposure Score'][i][j]) for j in range(I)) for i in range(I)) + \
              (yPlus[3]-yMinus[3]) == 19) for k in range(K))

# Constraint (5)
m.addConstrs(((sum(sum(z[k,i,j]) for j in range(I)) for i in range(I)) >= 5) for k in range(K)) 

# Constraint (6)
m.addConstrs(((sum(x[k,i]) for i in range(I)) <= 2) for k in range(K))

# Constraint (7)
m.addConstrs(((sum(x[k,i]) for i in range(I)) == 1) for k in range(K))

# Constraint (8)
m.addConstrs((sum(x[k,7]) == 1) for k in range(K))

# Update Model
m.update()

TypeError: unsupported operand type(s) for +: 'generator' and 'generator'

#### Step 7: Solve the model

In [ ]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

#### Step 8: Print variable values  (The Messy Way)

In [ ]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
print('\n')
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.z)) 
print('\n')
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.y))
    

#### Step 8 Alternate: Print the solution (The Easy To Read Way)

#### Optional Step: Exporting the LP File for Debugging
Export it to an LP file and check if it is in the desired shape.

In [ ]:
m.write("checkModel.lp")